# Lecture 2: Data Preparation

In this lesson you'll carry out some of the data cleaning steps required to prepare data for pretraining. In the video, Sung mentioned an Upstage tool called **Dataverse** which can help you with data cleaning. You can checkout the features of Dataverse at [this link](https://github.com/UpstageAI/dataverse).

In [73]:
import warnings
warnings.filterwarnings("ignore")

## 1. Sourcing datasets for pretraining

In this section, you'll see two ways to source data for training:
1. Download an existing dataset from Hugging Face
2. Create a dataset of python scripts sourced from Github

In both cases the result will be a Hugging Face `Dataset` object, part of the `Datasets` library. You can read more about the properties of Datasets and how to work with them on the [Hugging Face website](https://huggingface.co/docs/datasets/en/index).

### Download data from Hugging face

The dataset you download here is a subset of a much larger dataset called **Red Pajama**. The full, 1 trillion token dataset is available on Hugging Face at [this link](https://huggingface.co/datasets/togethercomputer/RedPajama-Data-1T).

In [74]:
import sys;
print(sys.executable)

import sys, importlib.util, subprocess, json
print("exe:", sys.executable)
print("find_spec:", importlib.util.find_spec("datasets"))

/Users/kichinosukey-mba/pretrainingLLMs/.venv/bin/python
exe: /Users/kichinosukey-mba/pretrainingLLMs/.venv/bin/python
find_spec: ModuleSpec(name='datasets', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1064d95d0>, origin='/Users/kichinosukey-mba/pretrainingLLMs/.venv/lib/python3.11/site-packages/datasets/__init__.py', submodule_search_locations=['/Users/kichinosukey-mba/pretrainingLLMs/.venv/lib/python3.11/site-packages/datasets'])


In [75]:
import datasets
pretraining_dataset = datasets.load_dataset(
    "togethercomputer/RedPajama-Data-V2",
    name="default",
    partition="head_middle",
    snapshots=["2023-06"],
    languages=["en"],
    streaming=True
)


In [76]:
print(pretraining_dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['raw_content', 'doc_id', 'meta', 'quality_signals'],
        n_shards: 10000
    })
})


Only work with the `text` column:

In [77]:
pretraining_dataset = pretraining_dataset.select_columns(
    ["raw_content"]
)

In [78]:
print(pretraining_dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['raw_content'],
        n_shards: 10000
    })
})


In [79]:
# raw_content を text にリネーム
pretraining_dataset = pretraining_dataset.rename_column("raw_content", "text")

In [80]:
# 先に欲しい件数だけ集める
max_samples = 10_000
rows = []
for i, row in enumerate(pretraining_dataset["train"]):
    rows.append(row)  # row は dict 形式
    if i + 1 >= max_samples:
        break

# Iterable → Dataset へ変換
pretraining_dataset = datasets.Dataset.from_list(rows)  # or Dataset.from_dict({k: [r[k] for r in rows] for k in rows[0]})
print(pretraining_dataset)  # -> Dataset

Dataset({
    features: ['text'],
    num_rows: 10000
})


### Compare pretraining and fine-tuning datasets
In the next cell, you'll download a fine-tuning dataset to contrast with the pretraining dataset you loaded above. You can read more about the Alpaca model and instruction tuning dataset [here](https://crfm.stanford.edu/2023/03/13/alpaca.html). 

In [81]:
instruction_dataset = datasets.load_dataset(
    "c-s-ale/alpaca-gpt4-data",
    split='train'
)
print(instruction_dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 52002
})


In [82]:
i=0
print("Instruction: " + instruction_dataset[i]["instruction"] 
      + "\nInput: " + instruction_dataset[i]["input"] 
      + "\nOutput: " + instruction_dataset[i]["output"])

Instruction: Give three tips for staying healthy.
Input: 
Output: 1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.

2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.

3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.


Notice how in contrast to the pretraining data, which is just raw text, fine-tuning datasets are structured into question-answer pairs or instruction-response sets that can include additional input context if required. 

Moving forward, you'll only work with the unstructured pretraining dataset.

### Scrape python code from Github
Here, you'll download a selection of python scripts from Github and then prepare them as a Hugging Face `Dataset` object to use in training. 

The same pattern here will work for preparing any text scraped from the web.

In [83]:
# Import some required packages
import os
import requests

# Path to directory to store python scripts
code_dir = "./code"

In [84]:
urls = [
    "https://raw.githubusercontent.com/TheAlgorithms/Python/master/searches/double_linear_search_recursion.py",
    "https://raw.githubusercontent.com/KosingZhu/tensorflow/master/tensorflow/python/tools/module_util.py",
    "https://raw.githubusercontent.com/EricRemmerswaal/tensorflow/master/tensorflow/python/distribute/distribute_coordinator_context.py",
    "https://raw.githubusercontent.com/computationalartist/tensorflow/master/tensorflow/python/ops/numpy_ops/integration_test/benchmarks/numpy_mlp.py",
    "https://raw.githubusercontent.com/Van-an/tensorflow/master/tensorflow/python/distribute/coordinator/values.py",
    "https://raw.githubusercontent.com/nkgwer/tensorflow/master/tensorflow/lite/tools/visualize.py",
    "https://raw.githubusercontent.com/gitblazer/youtube-dl/master/youtube_dl/version.py",
    "https://raw.githubusercontent.com/Joshua-Barawa/My-Photos/master/venv/lib/python3.8/site-packages/django/contrib/messages/__init__.py",
    "https://raw.githubusercontent.com/PaliC/pytorch/master/test/fx/test_subgraph_rewriter.py"
]

Retrieve the python scripts:

In [85]:
for url in urls:
    print(f"Working on url: {url}")
    response = requests.get(url)
    file_name = os.path.basename(url)
    file_path = os.path.join(code_dir, file_name)
    
    with open(file_path, "wb") as file:
        file.write(response.content)

Working on url: https://raw.githubusercontent.com/TheAlgorithms/Python/master/searches/double_linear_search_recursion.py
Working on url: https://raw.githubusercontent.com/KosingZhu/tensorflow/master/tensorflow/python/tools/module_util.py
Working on url: https://raw.githubusercontent.com/EricRemmerswaal/tensorflow/master/tensorflow/python/distribute/distribute_coordinator_context.py
Working on url: https://raw.githubusercontent.com/computationalartist/tensorflow/master/tensorflow/python/ops/numpy_ops/integration_test/benchmarks/numpy_mlp.py
Working on url: https://raw.githubusercontent.com/Van-an/tensorflow/master/tensorflow/python/distribute/coordinator/values.py
Working on url: https://raw.githubusercontent.com/nkgwer/tensorflow/master/tensorflow/lite/tools/visualize.py
Working on url: https://raw.githubusercontent.com/gitblazer/youtube-dl/master/youtube_dl/version.py
Working on url: https://raw.githubusercontent.com/Joshua-Barawa/My-Photos/master/venv/lib/python3.8/site-packages/djan

In [86]:
files = os.listdir(code_dir)
for file in files:
    print(file)

test_subgraph_rewriter.py
numpy_mlp.py
values.py
version.py
double_linear_search_recursion.py
__init__.py
visualize.py
module_util.py
distribute_coordinator_context.py


Concatenate scripts into a list:

In [87]:
code_dataset = []
for file in os.listdir(code_dir):
    code_dataset.append(
        {'text': open(os.path.join(code_dir, file), 'r').read()}
    )

Convert list to Hugging Face `Dataset` object:

In [88]:
code_dataset = datasets.Dataset.from_list(code_dataset)
print(code_dataset)

Dataset({
    features: ['text'],
    num_rows: 9
})


Combine the python code dataset with the pretraining dataset you downloaded above:

In [89]:
print(pretraining_dataset)
print(code_dataset)

Dataset({
    features: ['text'],
    num_rows: 10000
})
Dataset({
    features: ['text'],
    num_rows: 9
})


In [90]:
dataset = datasets.concatenate_datasets(
    [pretraining_dataset, code_dataset]
)
print(dataset[:5])

{'text': ['AMICUS ANTHOLOGIES, PART ONE (1965-1972)\nFebruary 23, 2017 Alfred Eaker Leave a comment\nWith Dr. Terror’s House of Horrors (1965, directed by Freddie Francis and written by Milton Subotsky) Amicus Productions (spearheaded by Subotsky and Max Rosenberg, who previously produced for Hammer and was a cousin to Doris Wishman) established itself as a vital competitor to Hammer Studios. Rather than imitating Hammer’s modernization of Gothic classics, Amicus developed its own niche with omnibus films. They were successful enough to be in full-fledged production for a decade, establishing a reputation as the go-to studio for horror anthologies. This, their introductory portmanteau film, clearly influenced by EC Comics, sets a pattern of to-be-expected unevenness. Still, Amicus installs themselves as a horror studio to be reckoned with, sparing no expense in procuring Hammer’s top actors: Peter Cushing (who would star in all but one of the Amicus anthologies) and Christopher Lee. Fo

## 2. Data cleaning

In the cells below, you'll carry out the following cleaning steps:
1. Filter out samples that are too short
2. Remove repetitions within a single text example
3. Remove duplicated documents
4. Quality filter to remove non-English texts 

### Remove examples that are too short

In [91]:
dataset.num_rows

10009

In [92]:
import heapq

def paragraph_length_filter(x):
    """Returns False iff a page has too few lines or lines are too short."""
    lines = x['text'].split('\n')
    if (
        len(lines) < 3
        or min(heapq.nlargest(3, [len(line) for line in lines])) < 3
    ):
        return False
    return True

In [93]:
dataset = dataset.filter(
    paragraph_length_filter,
    load_from_cache_file=False
)

Filter:   0%|          | 0/10009 [00:00<?, ? examples/s]

In [94]:
dataset.num_rows

8966

### Remove repeated text within training examples

Here you'll remove text repetitions within each example. 

In [95]:
def find_duplicates(paragraphs):
    """
    Use this function to find the number of repetitions 
    in the paragraphs.
    """
    unique_x = set()
    duplicate_chars = 0
    duplicate_elements = 0
    for element in paragraphs:
        if element in unique_x:
            duplicate_chars += len(element)
            duplicate_elements += 1
        else:
            unique_x.add(element)
    return duplicate_elements, duplicate_chars

In [96]:
import re

def paragraph_repetition_filter(x):
    """
    Returns False iff a page has too many repetitions.
    """
    text = x['text']
    paragraphs = re.compile(r"\n{2,}").split(text.strip())                # Split by paragraphs (2 or more newlines)
    paragraphs_duplicates, char_duplicates = find_duplicates(paragraphs)  # Find number of duplicates in paragraphs
    if paragraphs_duplicates / len(paragraphs) > 0.3:
        return False
    if char_duplicates / len(text) > 0.2:
        return False
    return True

In [97]:
dataset = dataset.filter(
    paragraph_repetition_filter,
    load_from_cache_file=False
)

Filter:   0%|          | 0/8966 [00:00<?, ? examples/s]

In [98]:
dataset.num_rows

8965

### Deduplication

In this section, you'll remove duplicate examples from the entire dataset (in contrast to the previous step where you were just looking for repeated text in each example.)

In [99]:
def deduplication(ds):
    def dedup_func(x):
        """Use this function to remove duplicate entries"""
        if x['text'] in unique_text:
            return False
        else:
            unique_text.add(x['text'])
            return True

    unique_text = set()

    ds = ds.filter(dedup_func, load_from_cache_file=False, num_proc=1)
    return ds

dataset = deduplication(dataset)

Filter:   0%|          | 0/8965 [00:00<?, ? examples/s]

In [100]:
dataset.num_rows

8965

### Quality filter - Language

Here you'll remove any text examples that are in a language other than English. The code here uses a language detection model called fastText. You can read about fastText [here](https://fasttext.cc/).

In [101]:
# !pip install fasttext

In [103]:
import urllib
from fasttext.FastText import _FastText

def english_language_filter(ds):
    # load language detection model
    model = _FastText('./models/upstage/L2_language_model.bin')
    
    def is_english(x):
        # Predict language of the text and probability
        language, score = model.predict(x['text'].replace("\n", ""))

        language = language[0].split("__")[2]
        return score > 0.4 and language == "en" # change code here if building a model in another language

    ds = ds.filter(is_english, load_from_cache_file=False, num_proc=1)
    return ds

dataset = english_language_filter(dataset)

Parameter 'function'=<function english_language_filter.<locals>.is_english at 0x3221fe480> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Filter:   0%|          | 0/8965 [00:00<?, ? examples/s]

In [104]:
dataset.num_rows

8963

## 3. Save the dataset to disk

Read more about the parquet data format [here](https://parquet.apache.org/).

In [107]:
file_path = "./data/preprocessed_dataset.parquet"
dataset.to_parquet(file_path)

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

48710013